In [1]:
import os
import numpy as np
import pandas as pd
import shutil
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
# reads excel

filepath = "./05-EPI-025 - Run 1.xlsx"

if os.path.exists(filepath):
    file = pd.ExcelFile(filepath)
else:
    print("the path does not exists")

In [6]:
# compiles all tabs

sheet_names = file.sheet_names

raw_data = pd.DataFrame()

for sheet in sheet_names:
    df = pd.read_excel(filepath, sheet_name=sheet, index_col=0, header=None).dropna(how="all")
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df.insert(0, "Sheet", sheet)
    
    raw_data = pd.concat([raw_data, df])
    
    del df

In [7]:
raw_data.head()

,Sheet,Name,RT (min),% Area,Base peak (m/z),Retention Time,Base Peak
0,A1,NaN,0.495,15.2,79.15,NaN,NaN
1,A1,NaN,3.471,0.22,233.38,NaN,NaN
2,A1,NaN,4.3,16.6,217.4,NaN,NaN
3,A1,E192,4.6,NaN,NaN,NaN,NaN
4,A1,NaN,4.603,27.72,302.42,NaN,NaN


In [8]:
raw_data.fillna(0, inplace=True)
raw_data["RT (min)"] = raw_data["RT (min)"] + raw_data["Retention Time"]
raw_data["Base peak (m/z)"] = raw_data["Base peak (m/z)"] + raw_data["Base Peak"]
raw_data.drop(["Retention Time", "Base Peak"], axis=1, inplace=True)

In [9]:
raw_data.head()

,Sheet,Name,RT (min),% Area,Base peak (m/z)
0,A1,0,0.495,15.20,79.15
1,A1,0,3.471,0.22,233.38
2,A1,0,4.300,16.60,217.40
3,A1,E192,4.600,0.00,0.00
4,A1,0,4.603,27.72,302.42


In [10]:
# renames & changes types

raw_data.columns = [i.strip() for i in raw_data.columns]

convert_dict = {
    'RT (min)': float, 
    'Base peak (m/z)': float,
    '% Area': float
} 

raw_data = raw_data.astype(convert_dict)

raw_data["Base peak (m/z)"] = raw_data["Base peak (m/z)"].round(0).astype(float)

In [11]:
# filters for area

area_cutoff = 0.1
raw_data_sel = raw_data.loc[raw_data["% Area"] >= area_cutoff]

In [12]:
# finds rt_range based on min_rt & max_rt

rt_interval = 0.05

min_rt = raw_data_sel["RT (min)"].min()
max_rt = raw_data_sel["RT (min)"].max()
rt_range = np.arange(min_rt, max_rt, rt_interval)

In [13]:
%%time
all_data = pd.DataFrame()

for rt in rt_range:
    rt_lower_bound = round(rt, 2)
    rt_upper_bound = round(rt + rt_interval, 2)
    rt_range_text = "{:.2f}-{:.2f}".format(rt_lower_bound, rt_upper_bound)
    
    df = raw_data_sel.loc[(raw_data_sel["RT (min)"] >= rt_lower_bound) & (raw_data_sel["RT (min)"] < rt_upper_bound)].copy()
    df["rt_range"] = rt_range_text
    df.reset_index(drop=True, inplace=True)
    df.set_index("Sheet", inplace=True)
    unique_base_peak = sorted(df["Base peak (m/z)"].unique())

    for base_peak in unique_base_peak:
        df2 = df.loc[df["Base peak (m/z)"] == base_peak, ["% Area"]]
        df2.columns = pd.MultiIndex.from_product([
            ["rt_range: {}".format(rt_range_text)],
            ["Base peak: {}".format(base_peak)]])
                
        df3 = df2.groupby(df2.index).sum()
        
        if all_data.shape[0] == 0:
            all_data = df3
        else:
            all_data = pd.merge(
                left=all_data,
                right=df3,
                left_index=True, 
                right_index=True,
                how = "outer")
        
        del df2, df3

    del df

CPU times: user 873 ms, sys: 6.74 ms, total: 880 ms
Wall time: 881 ms


In [14]:
# reorders indexes

new_indexes = [j[::-1] for j in sorted([i[::-1] for i in all_data.index])]
final_dataset = all_data.loc[new_indexes]

In [15]:
# saves to excel

filename, extension = os.path.splitext(filepath)
dest_filepath = "{}_res{}".format(filename, extension)

with pd.ExcelWriter(dest_filepath) as writer:
    final_dataset.to_excel(
        writer, 
        sheet_name="all_auto", 
        index=True, 
        freeze_panes=(3, 1),        
    )